In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torchvision import transforms
from pathlib import Path
from dataclasses import dataclass
import torch.nn.functional as F
from torch import nn
from utils import print_and_get_nb_trainable_parameters, print_trainable_parameters
import os
import keras

2023-12-14 14:20:18.702065: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-14 14:20:18.742683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 14:20:18.742719: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 14:20:18.743660: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 14:20:18.750108: I tensorflow/core/platform/cpu_feature_guar

In [2]:
@dataclass
class Config:
    data_dir = Path('data')
    train_dir = data_dir / 'train'
    validation_dir = data_dir / 'validation'
    image_shape = (3, 224, 224)
    image_size = (224, 224)
    num_workers = os.cpu_count()
    batch_size = 32


config = Config()

In [5]:
#create dataset using ImageFolder
train_transforms = transforms.Compose([
                                 transforms.Resize(config.image_size),
                                 transforms.RandomHorizontalFlip(p=0.5),
                                 transforms.RandomVerticalFlip(p=0.5),
                                #  transforms.RandomRotation(degrees=45),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                                 ])
validation_transforms = transforms.Compose([
                                    transforms.Resize(config.image_size),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                                    ])

In [6]:
train_dataset = ImageFolder(config.train_dir, transform=train_transforms)
validation_dataset = ImageFolder(config.validation_dir, transform=validation_transforms)
len(train_dataset), len(validation_dataset)

FileNotFoundError: [Errno 2] No such file or directory: 'data/train'

In [7]:
#create dataloaders

train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        pin_memory=True,
    )

val_dataloader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=config.num_workers,
    pin_memory=True,
)

NameError: name 'train_dataset' is not defined

In [5]:
class Net(nn.Module):
    def __init__(self, num_classes=5):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, padding='same'),
            nn.LeakyReLU(),
            nn.Conv2d(32, 32, kernel_size=5, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 64, kernel_size=3, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, kernel_size=3, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 512, kernel_size=3, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
        self.features.requires_grad_(False)

        print_trainable_parameters(self)
        print_and_get_nb_trainable_parameters(self)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
# config.class_names = train_dataset.classes
model = Net()

trainable params: 66,565 || all params: 4,062,181 || trainable%: 1.638651748900406
trainable params: 66,565 || all params: 4,062,181 || trainable%: 1.638651748900406


In [ ]:
class Net_1(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, padding='same'),
            nn.LeakyReLU(),
            nn.Conv2d(32, 32, kernel_size=5, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size='same'),  # 1x1 convolution
            nn.LeakyReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=1),  # 1x1 convolution
            nn.LeakyReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, 512, kernel_size=3, padding='same'),
            nn.LeakyReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.classifier = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
        # self.features.requires_grad_(False)

        # print_trainable_parameters(self)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
config.class_names = train_dataset.classes

In [ ]:
class KerasModel(keras.Model):
    keras.backend.clear_session()
    def __init__(self):
        super().__init__()
        self.model = keras.layers.TorchModuleWrapper(Net(len(config.class_names)), name='torch_model')
    
    def call(self, x):
        x = self.model(x)
        return x
    
keras_model = KerasModel()

In [ ]:
keras_model.summary()

Model: "keras_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ torch_model                     │ ?                         │  4,062,181 │
│ (TorchModuleWrapper)            │                           │            │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 4,062,181 (15.50 MB)

 Trainable params: 4,062,181 (15.50 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
keras_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adamW",
    metrics=["accuracy"]
)

In [ ]:
keras_model.fit(
    train_dataloader,
    epochs=10,
    validation_data=val_dataloader,
)

In [ ]:
import torch
from typing import Callable


def train_one_epoch(model: torch.nn.Module,
                    train_dataloader: torch.utils.data.DataLoader,
                    loss_fn,
                    optimizer: torch.optim.Optimizer,
                    scheduler: torch.optim.lr_scheduler,
                    device: torch.device,
                    f1score: Callable,
                    accuracy: Callable,
                    DEBUG: bool = False) -> tuple:
    """
    Trains the model for one epoch using the specified dataloader and returns the average training loss,
    accuracy and F1-score for the epoch.

    Args:
        model: The neural network model to be trained
        epoch: The current epoch number
        train_dataloader: DataLoader object containing the training data
        loss_fn: The loss function to be used for optimization
        optimizer: The optimization algorithm to be used
        scheduler: The learning rate scheduler to be used
        device: The device on which the computations should be carried out
        f1score: The F1-score metric to be used for evaluation
        accuracy: The accuracy metric to be used for evaluation
        DEBUG: A boolean flag to enable debug mode

    Returns:
        Tuple[float, float, float]: A tuple containing the average training loss, accuracy and F1-score for the epoch.
    """
    # Set the model to training mode.
    model.train()
    # Initialize the running totals for the loss, accuracy and F1-score.
    train_acc_total, train_loss_total, train_f1score_total = 0, 0, 0
    # Iterate over the training dataloader.
    for batch_idx, (X, y) in enumerate(train_dataloader):
        # If DEBUG is enabled, break after the first 10 batches.
        if DEBUG and batch_idx == 10:
            break
        # Move the data to the device.
        X, y = X.to(device), y.to(device)
        # Forward pass.
        y_pred = model(X)
        # Calculate the loss.
        loss = loss_fn(y_pred, y)
        # Backward pass.
        optimizer.zero_grad()
        loss.backward()
        # Update the parameters.
        optimizer.step()
        # Update the learning rate scheduler.
        scheduler.step()
        # Calculate the accuracy and F1-score.
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        f1score_metrics = f1score(y_pred_class, y)
        acc = accuracy(y_pred_class, y)
        # Print a progress message.
        msg = f"Batch: {batch_idx + 1} | Loss: {loss.item():.4f} | Accuracy: {acc:.4f} | F1Score: {f1score_metrics:.4f}"
        print('\r' + msg, end='', flush=True)
        # Update the running totals.
        train_loss_total = train_loss_total + loss.item()
        train_acc_total = train_acc_total + acc
        train_f1score_total = train_f1score_total + f1score_metrics
    # Calculate the average loss, accuracy and F1-score.
    train_loss_avg = train_loss_total / len(train_dataloader)
    train_acc_avg = train_acc_total / len(train_dataloader)
    train_f1score_avg = train_f1score_total / len(train_dataloader)
    # If DEBUG is enabled, calculate the average loss, accuracy and F1-score for the first 10 batches.
    if DEBUG:
        train_loss_avg = train_loss_total / 10
        train_acc_avg = train_acc_total / 10
        train_f1score_avg = train_f1score_total / 10
    # Print the epoch history.
    epoch_history = f"Loss: {train_loss_avg:.4f} | Accuracy: {train_acc_avg:.4f} | F1Score: {train_f1score_avg:.4f}"
    print('\r' + epoch_history, end='\n', flush=True)
    return train_loss_avg, train_acc_avg, train_f1score_avg


# function to validate the model for one epoch
def validate_one_epoch(model: torch.nn.Module,
                       val_dataloader: torch.utils.data.DataLoader,
                       loss_fn,
                       device: torch.device,
                       f1score: Callable,
                       accuracy: Callable,
                       DEBUG: bool = False) -> tuple:
    """
        Function to validate the model for one epoch.

        Parameters:
        model: The model to validate.
        val_dataloader: Dataloader for the validation set.
        loss_fn: Loss function used for validation.
        device: The device type used (GPU or CPU).
        f1score: Function to calculate the F1 score.
        accuracy: Function to calculate the accuracy.
        DEBUG: A boolean flag used for debugging. If true, only 10 batches will be validated.

        Returns:
        A tuple containing average validation loss, accuracy, and F1 score for the epoch.
    """
    # Switch model to evaluation mode
    model.eval()
    # Initialize counters for total validation loss, accuracy and F1 score
    val_acc_total, val_loss_total, val_f1score_total = 0, 0, 0
    # Disabling gradient calculation as we are in validation mode
    with torch.no_grad():
        # Loop through all batches in the validation dataloader
        for batch_idx, (X, y) in enumerate(val_dataloader):
            # Debug condition
            if DEBUG and batch_idx == 10:
                break
            # Move the batch tensors to the same device as the model
            X, y = X.to(device), y.to(device)
            # Forward pass
            y_pred = model(X)
            # Calculate the loss
            loss = loss_fn(y_pred, y)
            # Calculate the accuracy and F1-score
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            f1score_metrics = f1score(y_pred_class, y)
            acc = accuracy(y_pred_class, y)
            # Update the running totals
            val_loss_total = val_loss_total + loss.item()
            val_acc_total = val_acc_total + acc
            val_f1score_total = val_f1score_total + f1score_metrics
            # Print a progress message
            msg = f"Batch: {batch_idx + 1} | Validation Loss: {loss.item():.4f} | Validation Accuracy: {acc:.4f} | Validation F1Score: {f1score_metrics:.4f}"
            print('\r' + msg, end='', flush=True)
        # Calculate the average loss, accuracy and F1-score
        val_loss_avg = val_loss_total / len(val_dataloader)
        val_acc_avg = val_acc_total / len(val_dataloader)
        val_f1score_avg = val_f1score_total / len(val_dataloader)
        # If DEBUG is enabled, calculate the average loss, accuracy and F1-score for the first 10 batches.
        if DEBUG:
            val_loss_avg = val_loss_total / 10
            val_acc_avg = val_acc_total / 10
            val_f1score_avg = val_f1score_total / 10
        epoch_history = f"Validation Loss: {val_loss_avg:.4f} | Validation Accuracy: {val_acc_avg:.4f} | Validation F1Score: {val_f1score_avg:.4f}"
        print('\r' + epoch_history, end='\n', flush=True)
    return val_loss_avg, val_acc_avg, val_f1score_avg

In [ ]:
from engine import train_one_epoch, validate_one_epoch
import torch
from dataloaders import get_train_val_dataloader
from models import EfficientNet
from torchmetrics import Accuracy, F1Score
from config import get_args
import time
from utils import save_model
import wandb
from tqdm import tqdm
from discordutils import send_msg
from wandb_utils import log_values, initialize_wandb_run


def main(args):
    DEBUG = args.DEBUG
    if DEBUG:
        print("Debugging mode is on. Only 10 batches will be used for training and validation.")
    loss = torch.nn.CrossEntropyLoss()
    f1score = F1Score(task="multiclass", num_classes=25, average="macro").to(args.device)
    accuracy = Accuracy(task='multiclass', num_classes=25).to(args.device)
    model = EfficientNet()
    # model = torch.compile(model)
    train_dataloader, _, _ = get_train_val_dataloader(root_dir=args.data_dir,
                                                                         batch_size=args.batch_size)
    model = model.to(args.device)
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args.learning_rate)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=args.learning_rate, steps_per_epoch=len(
        train_dataloader), epochs=args.epochs)
    logs = train(args, args.device, loss, f1score, accuracy, model, optimizer, scheduler=scheduler)
    print(logs)


def train(args, device, loss, f1score, accuracy, model, optimizer, starting_epoch=0, scheduler=None):
    if args.wandb:
        run = initialize_wandb_run(args)
    print("Using train-test split.")
    DEBUG = args.DEBUG
    train_loss_list, train_acc_list, train_f1score_list = [], [], []
    val_loss_list, val_acc_list, val_f1score_list = [], [], []
    train_dataloader, val_dataloader, classes = get_train_val_dataloader(root_dir=args.data_dir,
                                                                         batch_size=args.batch_size)
    if args.wandb:
        run.watch(model, log="gradients", log_freq=100)
    for epoch in tqdm(range(starting_epoch, args.epochs + starting_epoch), disable=False, desc="Epochs"):
        start_time = time.time()
        print(f"\nEpoch: {epoch}")
        epoch_loss, epoch_acc, epoch_f1score = train_one_epoch(model, train_dataloader, loss, optimizer, scheduler, device,
                                                               f1score, accuracy, DEBUG)
        epoch_val_loss, epoch_val_acc, epoch_val_f1score = validate_one_epoch(model, val_dataloader, loss, device,
                                                                              f1score, accuracy, DEBUG)
        end_time = time.time()
        epoch_time = end_time - start_time
        print(f"Epoch time: {epoch_time:.2f} seconds")
        if args.wandb:
            log_values(run, step=epoch, loss=epoch_loss, f1score=epoch_f1score, accuracy=epoch_acc, val_loss=epoch_val_loss, val_f1score=epoch_val_f1score, val_accuracy=epoch_val_acc)
        train_loss_list.append(epoch_loss)
        train_acc_list.append(epoch_acc)
        train_f1score_list.append(epoch_f1score)
        val_loss_list.append(epoch_val_loss)
        val_acc_list.append(epoch_val_acc)
        val_f1score_list.append(epoch_val_f1score)
        # run.alert(title="Epoch completed", text=f"Epoch {epoch + 1} completed with F1score: {epoch_val_f1score:.4f}.")
        if args.discord:
            send_msg(f"Epoch {epoch} completed with F1score: {epoch_val_f1score:.4f}.")
        save_model(model_name="efficientnet",
                   model=model,
                   optimizer=optimizer,
                   epoch=epoch,
                   loss=epoch_loss,
                   val_f1score=epoch_val_f1score,
                   directory=args.model_dir)
    if args.wandb:
        run.finish()
    if args.discord:
        send_msg("Training complete")
    return train_loss_list, train_acc_list, train_f1score_list, val_loss_list, val_acc_list, val_f1score_list